<a href="https://colab.research.google.com/github/XiaoleiZ/Reproduce_HilbertCNN/blob/master/Demo_HibertCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implement HilbertCNN

Paper: [An Image representation based on convolutional network for DNA classification](https://arxiv.org/pdf/1806.04931.pdf)

Method Summary: Transform 1D-DNA sequence into 2D image to improve the classification performance of using CNN

Steps: 

1.Transformation from 1D sequence to 2D image

2.CNN classification network



# DNA sequence representation

1.Represent a sequence as a list of k-mers

2.Transform each k-mer into a one-hot vector
-> sequence is represented as a list of one-hot vector

3.Use Hilbert Curves to assign each element of the list of k-mers to a pixel in an image

In [0]:
import numpy as np
import keras
from keras import Sequential,Model
from keras.layers import Activation, Dense,Dropout,Input,Conv2D, BatchNormalization,AveragePooling2D,Flatten
from keras.optimizers import Adam
from keras import backend as K
import numpy as np
from sklearn.model_selection import train_test_split
import urllib
import re



In [0]:
def kmerize(sequence, k=4):
    
    ###Description: represent a sequence as list of k-mers
    ###Input: sequence (string), 
    ###       parameter k (k>=1), use 4 as default as shown in the paper.
    ###Output: a list of k-mer strings (array)
    
    #include the list of k-mers
    if k <1 or not int(k):
      raise ValueError("k must be an integer larger than zero");
    
    kmer_list = [];
    
    l=len(sequence);
    for i in range(0,l-k+1): # 0...l-k, the last kmer is the segment between l-k,l
      kmer_list.append(sequence[i:i+k]);
    
    return kmer_list;

In [0]:
def create_four_mer_ohv(k=4):
  
  ###Description: create a dictionary to store the one-hot-vector encoding (value) for each 4-mer (key)
  
  four_mer_list=[];
  base = ['A','C','G','T'];
  for n1 in base:
    for n2 in base:
      for n3 in base:
        for n4 in base:
          four_mer_list.append(n1+n2+n3+n4);
  
  four_mer_dict ={};
  l=len(four_mer_list);
  for i,x in enumerate(four_mer_list):
    vector=np.zeros(l);
    vector[i]=1;
    four_mer_dict[x]=vector;
    
  return four_mer_dict;
  

In [0]:
def one_hot_vectorize(kmer_list,four_mer_dict):
  
  ###Description: transfrom a list of k-mers into a list of one-hot-vectors
  ###Input: a list of k-mers representing a sequence
  ###       a dictionary created beforehand to store the one-hot-vector encoding for each possible kmer
  ###       (4-mer is used here)
  ###Output: a list of one-hot-encoding vectors representing the sequence
  return list(map(lambda x: four_mer_dict[x], kmer_list))


In [0]:
def hilbert_curve (n):
    #Description: Implementation of Hilbert space filling curve of order log2(n) 
    #             (number of cells: n*n) to map a distance in 1-D to coordinate in 2D array
    #Input: the number of cells in each dimension
    #return: 2D array - for each coordinate x,y, the value is the position in 1D the hlbert map corresponds to.
    #
    if n==2:
        return np.array([[0,3],[1,2]],int)
    unit=hilbert_curve(int(n/2));
    #the number of cells in each quardrant
    step = int(n**2/4)
    #the directions of the four quardrant given in the paper
    ########################
    ### a # d #
    ###   #   # 
    ########################
    ### b # c #
    ###   #   #
    ########################
    
    #for the first quadrant: anti-closewise rotate 90 degree
    a = np.flipud(np.rot90(unit));
    b = unit +step
    c = unit +step*2
    d = np.fliplr(np.rot90(unit)) +step*3
    #stack the four quardrant together
    new = np.concatenate((np.concatenate((a,b),axis=0),np.concatenate((d,c),axis=0)),axis=1)
    return new  

In [0]:
def pixelize_seqs(seqs,k=4):
  #Description: give a batch of sequence, process these sequences into images
  #Input: a list of sequences, each element is a string
  #Ouput: a list of images, each element is an image corresponding to the sequence
  
  #crop the hilbert curve
  curve=hilbert_curve(2**5)[:,0:16]
  n_height,n_width,n_depth = curve.shape+(256,)
  
  img_list = np.zeros((len(seqs),n_height,n_width,n_depth))
  
  for i in range(len(seqs)):
    kmer_list=kmerize(seqs[i],k);
    four_mer_dict=create_four_mer_ohv(k)
    ohv_list=one_hot_vectorize(kmer_list,four_mer_dict)
    for j in range(len(ohv_list)):
    #find the coordinate in the img
      img_list[i,np.where(curve==j)]=ohv_list[j]
  
  return img_list

In [0]:
def read_data(file):
  if re.search("^http", file):
    f = urllib.request.urlopen(file)
  else: 
    f=open(file)
  content = f.readlines()
  line=1
  name=[]
  seq=[]
  label=[]
  for x in content:
    x=x.strip()
    x=x.decode('utf-8')
    if line % 3 == 1:
      name = name + [x[1:]] #remove ">"
    elif line % 3 == 2:
      seq = seq + [x]
    elif line % 3 == 0:
      label = label + [x]
    line += 1
  
  return name,seq,label

def one_hot_vector(label):
  '''
  Description: given a vector of label (either 0 or 1), transform it into one-hot vector encoding
  
  '''
  label_class = np.zeros((len(label),2))
  for i in range(len(label)):
    if label[i]:
      label_class[i][0]=1
    else:
      label_class[i][1]=1
  
  return label_class

#Define the architecture  of Convolutional Neural Network  

Captured from the original paper. 

![Hilbert CNN architecture](https://raw.githubusercontent.com/XiaoleiZ/Reproduce_HilbertCNN/master/HilbertCNN_architecture.png)

In [0]:
def res_block(inputs,k1,k2,link,out):
  #residual block include: conv,bn,af,conv,bn
  #
  x=inputs
  x=Conv2D(filters=link,kernel_size=(k1,k1),padding='same',input_shape=inputs.shape)(x)
  x=BatchNormalization()(x)
  x=Activation('elu')(x)
  x=Conv2D(filters=out,kernel_size=(k2,k2),padding='same')(x)
  x=BatchNormalization()(x)
  
  return x


def Computational_block(input_shape,k1,k2,k3,k4,link,out):
  #Description: build the computational block: 
  #first the sum of two residul block and one identitiy map 
  #BatchNormalization->Activation Function
  
  #Input:
  #input_shape
  #k1, k2 - respective filter size of the first and second conv layers in the first residual block
  #k3, k4 - respective filter size of the first and second conv layers in the second residual block 
  #link - dimension of output of the first conv layer
  #out - dimension of output of the residual block
  
  #two residual blocks
  inputs=Input(input_shape)
  x1=res_block(inputs,k1,k2,link,out)
  x2=res_block(inputs,k3,k4,link,out)
  if input_shape[-1]==out:
    x=keras.layers.add([inputs,x1,x2])
  else:
    x_input=Conv2D(filters=out,kernel_size=(1,1),padding='same',input_shape=inputs.shape)(inputs)
    x=keras.layers.add([x_input,x1,x2])
  x=BatchNormalization()(x)
  x=Activation('elu')(x)
  
  model=Model(inputs=inputs,outputs=x)
  
  return model
  
  
def HCNN():
  '''
  Input: 32*16*256 images
  (1) Conv1: 7*7 conv, BN -> (output size) 32*16*64
  (2) Conv2: 5*5 conv, BN -> (output size)
  '''
  model = Sequential()
  model.add(Conv2D(filters=64,kernel_size=(7,7),padding='same',input_shape=(32,16,256)))
  model.add(BatchNormalization())
  model.add(Conv2D(filters=64,kernel_size=(5,5),padding='same',input_shape=(32,16,64)))
  model.add(BatchNormalization())
  model.add(Activation('elu'))
  model.add(AveragePooling2D(pool_size=(2,2),strides=2))
  model.add(Computational_block(input_shape=[16,8,64],k1=8,k2=4,k3=4,k4=3,link=4,out=32))
  model.add(Computational_block(input_shape=[16,8,32],k1=3,k2=3,k3=3,k4=3,link=4,out=32))
  model.add(AveragePooling2D(pool_size=(2,2),strides=2))
  model.add(Computational_block(input_shape=[8,4,32],k1=2,k2=4,k3=4,k4=3,link=4,out=32))
  model.add(Computational_block(input_shape=[8,4,32],k1=2,k2=2,k3=2,k4=2,link=4,out=32))
  model.add(Computational_block(input_shape=[8,4,32],k1=3,k2=2,k3=2,k4=3,link=4,out=32))
  model.add(AveragePooling2D(pool_size=(2,2),strides=2))
  model.add(BatchNormalization())
  model.add(Activation('elu'))
  model.add(AveragePooling2D(pool_size=(2,2),strides=2))
  #the first classification layer
  model.add(Flatten())
  model.add(Activation('elu')) #mentioned in the paper but not in the code
  model.add(Dropout(rate=0.5))
  #the second classification layer
  model.add(Dense(100))
  model.add(Activation('elu'))
  model.add(Dropout(rate=0.5))
  #the third classification layer
  model.add(Dense(50))
  model.add(Activation('elu'))
  model.add(Dropout(rate=0.5))
  model.add(Dense(2,activation='softmax'))
    
  return model

## Load the data

In [71]:
#Read the data

demo_file_url = 'https://raw.githubusercontent.com/XiaoleiZ/Reproduce_HilbertCNN/master/data/H4_1000.txt'
name,seq,label=read_data(demo_file_url)
imgs=pixelize_seqs(seq,k=4)
label = np.reshape(np.array(list(map(int,label))),(len(label),1))
label = one_hot_vector(label)
# check the dimension
print("Transformed image shape:"+str(imgs.shape))
print("Label class shape:"+str(label.shape))


Transformed image shape:(1000, 32, 16, 256)
Label class shape:(1000, 2)


## Split the data into training (2/3) and test (1/3) dataset

In [72]:
## train, test data split
X_train, X_test, y_train, y_test = train_test_split(
imgs, label, test_size=0.33, random_state=42)

print("Training data feature shape:"+str(X_train.shape))
print("Test data feature shape:"+str(X_test.shape))
print("Training data label shape:"+str(y_train.shape))
print("Test data label shape:"+str(y_test.shape))


Training data feature shape:(670, 32, 16, 256)
Test data feature shape:(330, 32, 16, 256)
Training data label shape:(670, 2)
Test data label shape:(330, 2)


In [0]:
hcnn=HCNN()
hcnn.compile(optimizer=Adam(lr=0.003),loss='categorical_crossentropy',metrics=['accuracy'])


## Network summary

In [74]:
hcnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_70 (Conv2D)           (None, 32, 16, 64)        802880    
_________________________________________________________________
batch_normalization_85 (Batc (None, 32, 16, 64)        256       
_________________________________________________________________
conv2d_71 (Conv2D)           (None, 32, 16, 64)        102464    
_________________________________________________________________
batch_normalization_86 (Batc (None, 32, 16, 64)        256       
_________________________________________________________________
activation_61 (Activation)   (None, 32, 16, 64)        0         
_________________________________________________________________
average_pooling2d_13 (Averag (None, 16, 8, 64)         0         
_________________________________________________________________
model_16 (Model)             (None, 16, 8, 32)         26248     
__________

## Training and testing: use 128 samples in each batch and 10 as the epoch size

In [0]:
hcnn.fit(X_train,y_train,batch_size=128,epochs=10,verbose=1,shuffle=True,validation_data=(X_test,y_test))
score = hcnn.evaluate(X_test,y_test,verbose=0)
print('Test accuracy:',score[1])

Train on 670 samples, validate on 330 samples
Epoch 1/10
670/670 [==============================] - 66s 99ms/step - loss: 0.9668 - acc: 0.4821 - val_loss: 0.7461 - val_acc: 0.5697
Epoch 2/10
670/670 [==============================] - 55s 82ms/step - loss: 0.8587 - acc: 0.5239 - val_loss: 0.8488 - val_acc: 0.4303
Epoch 3/10
670/670 [==============================] - 54s 81ms/step - loss: 0.8281 - acc: 0.5761 - val_loss: 0.8667 - val_acc: 0.4303
Epoch 4/10
256/670 [==========>...................] - ETA: 29s - loss: 0.7699 - acc: 0.5742